<a id='top'></a>
# Challenge Set 9
Topic:        Classification  
Date:         8/8/2016  
Name:         Whitney Chia  
Worked with:  All by myyyyyyyself

[Question 1](#question1)   
[Question 2](#question2)   
[Question 3](#question3)  
[Question 4](#question4)   
[Question 5](#question5)   
[Question 6](#question6)

<a id='question1'></a>
# Question 1   
[Top](#top)

In [91]:
# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
pd.options.display.max_rows = 999
plt.style.use('ggplot')
sns.set_context('poster')
sns.set_style('darkgrid')
sns.set_palette('dark')
import matplotlib.patches as mpatches
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation as cv
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [33]:
from pandas.io.stata import StataReader
reader = StataReader('ships.dta')
df = reader.data()

In [34]:
df.head()

,type,construction,operation,months,damage
0,A,1960-64,1960-74,127.0,0.0
1,A,1960-64,1975-79,63.0,0.0
2,A,1965-70,1960-74,1095.0,3.0
3,A,1965-70,1975-79,1095.0,4.0
4,A,1970-74,1960-74,1512.0,6.0


In [35]:
df['type'].unique()

[A, B, C, D, E]
Categories (5, object): [A < B < C < D < E]

In [36]:
df['1960_64_con'] = [1 if x == '1960-64' else 0 for x in df['construction']]
df['1965-70_con'] = [1 if x == '1965-70' else 0 for x in df['construction']]
df['1970_74_con'] = [1 if x == '1970-74' else 0 for x in df['construction']]
df['1975_79_con'] = [1 if x == '1975-79' else 0 for x in df['construction']]

In [37]:
df['1960_74_op'] = [1 if x == '1960-74' else 0 for x in df['operation']]
df['1975_79_op'] = [1 if x == '1975-79' else 0 for x in df['operation']]
df['constant'] = [1 for x in df['operation']]

In [38]:
df['A'] = [1 if x == 'A' else 0 for x in df['type']]
df['B'] = [1 if x == 'B' else 0 for x in df['type']]
df['C'] = [1 if x == 'C' else 0 for x in df['type']]
df['D'] = [1 if x == 'D' else 0 for x in df['type']]
df['E'] = [1 if x == 'E' else 0 for x in df['type']]

In [132]:
df.head()

,type,construction,operation,months,damage,1960_64_con,1965-70_con,1970_74_con,1975_79_con,1960_74_op,1975_79_op,constant,A,B,C,D,E
0,A,1960-64,1960-74,127.0,0.0,1,0,0,0,1,0,1,1,0,0,0,0
1,A,1960-64,1975-79,63.0,0.0,1,0,0,0,0,1,1,1,0,0,0,0
2,A,1965-70,1960-74,1095.0,3.0,0,1,0,0,1,0,1,1,0,0,0,0
3,A,1965-70,1975-79,1095.0,4.0,0,1,0,0,0,1,1,1,0,0,0,0
4,A,1970-74,1960-74,1512.0,6.0,0,0,1,0,1,0,1,1,0,0,0,0


In [40]:
Y, X = np.ravel(df['damage']), df[['months', '1960_64_con', '1965-70_con', '1970_74_con',\
                                               '1975_79_con', '1960_74_op', '1975_79_op', 'constant',\
                                              'A', 'B', 'C', 'D', 'E']]

In [41]:
pois_m = sm.GLM(Y,X, family=sm.families.Poisson(sm.families.links.log))
pois_results=pois_m.fit()

print pois_results.summary()

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   34
Model:                            GLM   Df Residuals:                       24
Model Family:                 Poisson   Df Model:                            9
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -84.182
Date:                Tue, 09 Aug 2016   Deviance:                       70.498
Time:                        15:51:26   Pearson chi2:                     65.8
No. Iterations:                     9                                         
                  coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
months       6.697e-05   8.52e-06      7.857      0.000      5.03e-05  8.37e-05
1960_64_con    -0.7190      0.161     -4.479     

It's very hard to tell if this is a good model or not without any context. Looking at the deviance and the Pearson chi-squared gives us a number but without looking at other models, those numbers have no context. We can't say this is a good model or a bad model without being able to compare it to another model. I would say it's an ok model since the deviance and the pearson aren't gigantic, but without a comparison, you can't really draw too many conclusions from those.

# <a id='question2'></a>
# Question 2   
[Top](#top)

In [45]:
Y, X = np.ravel(df['damage']), df[['1960_64_con', '1965-70_con', '1970_74_con',\
                                               '1975_79_con', '1960_74_op', '1975_79_op', 'constant',\
                                              'A', 'B', 'C', 'D', 'E']]
months = df['months']

In [46]:
pois_m = sm.GLM(Y,X, offset=np.log(months), family=sm.families.Poisson(sm.families.links.log))
pois_results=pois_m.fit()

print pois_results.summary()

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   34
Model:                            GLM   Df Residuals:                       25
Model Family:                 Poisson   Df Model:                            8
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -68.281
Date:                Tue, 09 Aug 2016   Deviance:                       38.695
Time:                        15:52:25   Pearson chi2:                     42.3
No. Iterations:                     9                                         
                  coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
1960_64_con    -1.2509      0.118    -10.617      0.000        -1.482    -1.020
1965-70_con    -0.5538      0.091     -6.110     

Using the offset clearly makes this model better, the deviance and the pearson chi2 have gotten smaller compared to the model without the offset. I believe the offset makes and adjustment for the time intervals so that they change to a rate, using that seems to be more effective for our model. 

# <a id='question3'></a>
# Question 3   
[Top](#top)

In [64]:
Y, X = np.ravel(df['damage']), df[['months', '1960_64_con', '1965-70_con', '1970_74_con',\
                                               '1975_79_con', '1960_74_op', '1975_79_op', 'constant',\
                                              'A', 'B', 'C', 'D', 'E']]

In [65]:
x_train, x_test, y_train, y_test = cv.train_test_split(X,Y,test_size=.30, random_state=4444)

months_train = x_train['months']
months_test = x_test['months']

x_train = x_train[['1960_64_con', '1965-70_con', '1970_74_con',\
                                               '1975_79_con', '1960_74_op', '1975_79_op', 'constant',\
                                              'A', 'B', 'C', 'D', 'E']]
x_test = x_test[['1960_64_con', '1965-70_con', '1970_74_con',\
                                               '1975_79_con', '1960_74_op', '1975_79_op', 'constant',\
                                              'A', 'B', 'C', 'D', 'E']]



Ask about getting the offset into this. I can't since I am slicing from my X and Y that I defined.

In [67]:
pois_m = sm.GLM(y_train, x_train, offset=np.log(months_train), family=sm.families.Poisson(sm.families.links.log))
pois_results = pois_m.fit()
pois_test = pois_results.predict(x_test)

In [68]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [133]:
print 'The mean squared error is: ' + str(mean_squared_error(y_test, pois_test))
print 'The mean absolute error is: ' + str(mean_absolute_error(y_test, pois_test))

The mean squared error is: 189.576620543
The mean absolute error is: 8.17832847596


# <a id='question4'></a>
# Question 4   
[Top](#top)

In [83]:
from statsmodels.genmod.generalized_linear_model import GLMResults

In [84]:
Y, X = np.ravel(df['damage']), df[['1960_64_con', '1965-70_con', '1970_74_con',\
                                               '1975_79_con', '1960_74_op', '1975_79_op', 'constant',\
                                              'A', 'B', 'C', 'D', 'E']]
months = df['months']

In [85]:
pois_m = sm.GLM(Y,X, offset=np.log(months), family=sm.families.Poisson(sm.families.links.log))
pois_results=pois_m.fit()

print pois_results.summary()

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   34
Model:                            GLM   Df Residuals:                       25
Model Family:                 Poisson   Df Model:                            8
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -68.281
Date:                Tue, 09 Aug 2016   Deviance:                       38.695
Time:                        16:04:42   Pearson chi2:                     42.3
No. Iterations:                     9                                         
                  coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
1960_64_con    -1.2509      0.118    -10.617      0.000        -1.482    -1.020
1965-70_con    -0.5538      0.091     -6.110     

In [86]:
Y, X = np.ravel(df['damage']), df['constant']

In [88]:
pois_m = sm.GLM(Y,X, family=sm.families.Poisson(sm.families.links.log))
pois_results_null=pois_m.fit()

print pois_results_null.summary()

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   34
Model:                            GLM   Df Residuals:                       33
Model Family:                 Poisson   Df Model:                            0
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -356.20
Date:                Tue, 09 Aug 2016   Deviance:                       614.54
Time:                        16:04:49   Pearson chi2:                     780.
No. Iterations:                     8                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
constant       2.3486      0.053     44.313      0.000         2.245     2.452


Not 100% sure how to interpret this but I would think that we can reject the null hypothesis because our model performs so much better than the null model based on deviance and pearson chi2.

# <a id='question5'></a>
# Question 5   
[Top](#top)

In [143]:
df['logY'] = [np.log(x +.1) for x in df['damage']]

In [144]:
Y, X = np.ravel(df['logY']), df[['months', '1960_64_con', '1965-70_con', '1970_74_con',\
                                               '1975_79_con', '1960_74_op', '1975_79_op', 'constant',\
                                              'A', 'B', 'C', 'D', 'E']]

In [145]:
OLS = sm.GLM(Y,X, family=sm.families.Gaussian(sm.families.links.log))
OLS_results = OLS.fit()

print OLS_results.summary()

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   34
Model:                            GLM   Df Residuals:                       24
Model Family:                Gaussian   Df Model:                            9
Link Function:                    log   Scale:                   2.89974044179
Method:                          IRLS   Log-Likelihood:                -61.342
Date:                Tue, 09 Aug 2016   Deviance:                       69.594
Time:                        17:36:22   Pearson chi2:                     69.6
No. Iterations:                   100                                         
                  coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
months       7.487e-05   3.21e-05      2.335      0.020       1.2e-05     0.000
1960_64_con    -1.5312      0.704     -2.175     

These definitely do not perform similarly, the deviance and the Pearson chi2 are higher than our poisson with an offset. Additionally you will notice that our P-values are all higher for the linear regression which would suggest model fits very poorly. Our p-values for the poisson with the offset showed that our features were actually significant whereas this one does not suggest that at all.

# <a id='question6'></a>
# Question 6   
[Top](#top)

In [101]:
reader = StataReader('smoking.dta')
df2 = reader.data()

In [103]:
df2.head()

,age,smoke,pop,dead
0,40-44,Doesn't smoke,656.0,18.0
1,45-49,Doesn't smoke,359.0,22.0
2,50-54,Doesn't smoke,249.0,19.0
3,55-59,Doesn't smoke,632.0,55.0
4,60-64,Doesn't smoke,1067.0,117.0


In [107]:
df2.columns

Index([u'age', u'smoke', u'pop', u'dead'], dtype='object')

In [105]:
df2['smoke'].unique()

[Doesn't smoke, Smokes cigars or pipe only, Smokes cigarettes and cigar or pipe, smokes cigarettes only]
Categories (4, object): [Doesn't smoke < Smokes cigars or pipe only < Smokes cigarettes and cigar or pipe < smokes cigarettes only]

In [110]:
df2['40-44_age'] = [1 if x == '40-44' else 0 for x in df2['age']]
df2['45-49_age'] = [1 if x == '45-49' else 0 for x in df2['age']]
df2['50-54_age'] = [1 if x == '50-54' else 0 for x in df2['age']]
df2['55-59_age'] = [1 if x == '55-59' else 0 for x in df2['age']]
df2['60-64_age'] = [1 if x == '60-64' else 0 for x in df2['age']]
df2['65-69_age'] = [1 if x == '65-69' else 0 for x in df2['age']]
df2['70-74_age'] = [1 if x == '70-74' else 0 for x in df2['age']]
df2['75-79_age'] = [1 if x == '75-79' else 0 for x in df2['age']]
df2['80+_age'] = [1 if x == '80+' else 0 for x in df2['age']]
df2["Doesn't smoke"] = [1 if x == "Doesn't smoke" else 0 for x in df2['smoke']]
df2['Smokes cigars or pipe only'] = [1 if x == 'Smokes cigars or pipe only' else 0 for x in df2['smoke']]
df2['Smokes cigarettes and cigar or pipe'] = [1 if x == 'Smokes cigarettes and cigar or pipe' else 0 for x in df2['smoke']]
df2['smokes cigarettes only'] = [1 if x == 'smokes cigarettes only' else 0 for x in df2['smoke']]

In [112]:
df2.columns

Index([                                u'age',
                                     u'smoke',
                                       u'pop',
                                      u'dead',
                                 u'40-44_age',
                                 u'45-49_age',
                                 u'50-54_age',
                                 u'55-59_age',
                                 u'60-64_age',
                                 u'65-69_age',
                                 u'70-74_age',
                                 u'75-79_age',
                                   u'80+_age',
                             u'Doesn't smoke',
                u'Smokes cigars or pipe only',
       u'Smokes cigarettes and cigar or pipe',
                    u'smokes cigarettes only'],
      dtype='object')

In [126]:
Y, X = np.ravel(df2['dead']), df2[[ '40-44_age', '45-49_age', '50-54_age', '55-59_age', '60-64_age', '65-69_age',\
                                '70-74_age', '75-79_age', '80+_age', "Doesn't smoke", 'Smokes cigars or pipe only',\
                                 'Smokes cigarettes and cigar or pipe','smokes cigarettes only']]
offset = df2['pop']

In [128]:
pois_m = sm.GLM(Y,X, offset=np.log(offset), family=sm.families.Poisson(sm.families.links.log))
pois_results=pois_m.fit()

print pois_results.summary()

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   36
Model:                            GLM   Df Residuals:                       24
Model Family:                 Poisson   Df Model:                           11
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -130.75
Date:                Tue, 09 Aug 2016   Deviance:                       21.487
Time:                        17:21:54   Pearson chi2:                     20.6
No. Iterations:                     9                                         
                                          coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------------------
40-44_age                              -2.1754      0.055    -39.643      0.000  

In [129]:
OLS = sm.GLM(Y,X, family=sm.families.Gaussian(sm.families.links.log))
OLS_results = OLS.fit()

print OLS_results.summary()

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   36
Model:                            GLM   Df Residuals:                       24
Model Family:                Gaussian   Df Model:                           11
Link Function:                    log   Scale:                   5836.21677323
Method:                          IRLS   Log-Likelihood:                -201.17
Date:                Tue, 09 Aug 2016   Deviance:                   1.4007e+05
Time:                        17:21:58   Pearson chi2:                 1.40e+05
No. Iterations:                    13                                         
                                          coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------------------------
40-44_age                               0.6480      0.370      1.749      0.080  

Again, our poisson performs better from both a deviance and a Pearson chi2 standpoint. Additionally our p-values for the poisson are all very small whereas the linear regression has some features that aren't contributing as much as they are in the poisson distribution.